In [9]:
import pandas as pd
import numpy as np
import json
import os 
import time
import torch.nn as nn
import torch
import transformers
from transformers import get_linear_schedule_with_warmup
from torch.nn.parallel import DataParallel

### Change the path here

In [2]:
training_filepath = '../../Data/BioASQ_data/BioASQ-training7b/trainining7b.json'
test_directory = '../../Data/BioASQ_data/Task7BGoldenEnriched/'

In [3]:
training_text = []
test_text = []

# Filter out the training data
with open (training_filepath, "r") as f:
    data = json.loads(f.read())
    texts = data['questions']
    for text in texts:
        if 'exact_answer' in text.keys():
            if text['exact_answer'] == 'yes' or text['exact_answer'] == 'no':
                training_text.append(text)

# Filter out the text data
directory = test_directory
for filename in os.listdir(directory):
    with open (directory+filename, "r") as f:
        data = json.loads(f.read())
        texts = data['questions']
        for text in texts:
            if 'exact_answer' in text.keys():
                if text['exact_answer'] == 'yes' or text['exact_answer'] == 'no':
                    test_text.append(text)

In [4]:
len(test_text)

140

In [5]:
def process_data(texts):
    question_list = []
    context_list = []
    target_list = []
    for text in texts:
        question_list.append(text['body'])
        context_list.append(' '. join([x['text'] for x in text['snippets']]))
        target_list.append(1 if text['exact_answer'] == 'yes' else 0)
    df = pd.DataFrame(zip(question_list, context_list, target_list), columns=['question', 'context', 'target'])
    return df

In [6]:
class BERTBaseUncased(nn.Module):
    def __init__(self, model_path):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(model_path)
        self.out = nn.Linear(768, 2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, ids, mask, token_type_ids):
        _, output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output = self.out(output)
        return self.softmax(output)

class BERTDatasetTraining:
    def __init__(self, question, context, target, tokenizer, max_len):
        self.question = question
        self.context = context
        self.target = target
        
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.question)
    
    def __getitem__(self, item):
        question= str(self.question[item])
        context = str(self.context[item])

        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True
        )

        ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']
        
        padding_len = self.max_len - len(ids)
        
        ids = ids[:self.max_len] + ([0] * padding_len) 
        token_type_ids = token_type_ids[:self.max_len] + ([0] * padding_len)
        mask = mask[:self.max_len] + ([0] * padding_len)
        

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.target[item], dtype=torch.long)
        }


def loss_fn(outputs, target):
    return nn.CrossEntropyLoss()(outputs, target)


def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d['ids']
        mask = d['mask']
        token_type_ids = d['token_type_ids']
        target = d['target']
        
        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        target = target.to(device, dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, target)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        if bi % 20 == 0:
            print(f'bi={bi}, loss={loss}')

            
def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        with torch.no_grad():
            ids = d['ids'].to(device, dtype=torch.long)
            mask = d['mask'].to(device, dtype=torch.long)
            token_type_ids = d['token_type_ids'].to(device, dtype=torch.long)
            target = d['target'].to(device, dtype=torch.long)
          
            outputs = model(ids, mask, token_type_ids)
          
            fin_targets.append(target.cpu().detach().numpy())
            fin_outputs.append(outputs.cpu().detach().numpy())

    return np.vstack(fin_outputs), np.hstack(fin_targets)

In [7]:
model_paths = [
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/8GB-checkpoints/run_8GB_model-trained-0-7063/',
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/8GB-checkpoints/run_8GB_model-trained-8-63567/',
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/8GB-checkpoints/run_8GB_model-trained-22-162449/'
              ]

In [8]:
for path in model_paths:
    try:
        print(path.split('/')[-2].split('.')[0])
    except Exception as e:
        print(e) 
        continue

run_8GB_model-trained-0-7063
run_8GB_model-trained-8-63567
run_8GB_model-trained-22-162449


In [10]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 4
EPOCHS = 8
LEARNING_RATE = 3e-5
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

start = time.perf_counter()

for model_path in model_paths:
    tokenizer_path = ('bert-base-uncased' if model_path == 'bert-base-uncased'\
                      else '../../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt')
    model_name = model_path if model_path == 'bert-base-uncased' else model_path.split('/')[-2].split('.')[0]
    scores = []
    model_stats = {'model_name':model_name,
                   'seeds':[],
                   'batch_size':TRAIN_BATCH_SIZE,
                   'epochs':EPOCHS,
                   'metric':'accuracy',
                   'scores': [],
                   'mean_score':0
                    }
    for num, seed in enumerate([42,43,44,45,46], 1):
      
        tokenizer = transformers.BertTokenizer.from_pretrained(tokenizer_path)
        model = BERTBaseUncased(model_path)
        model = DataParallel(model)
        model.to(device)

        train_df = process_data(training_text)
        test_df = process_data(test_text)

        train_dataset = BERTDatasetTraining(
            question=train_df.question.values,
            context=train_df.context.values,
            target=train_df.target.values,
            tokenizer=tokenizer,
            max_len=MAX_LEN
        )
        train_data_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=TRAIN_BATCH_SIZE,
            shuffle=True
        )

        test_dataset = BERTDatasetTraining(
            question=test_df.question.values,
            context=test_df.context.values,
            target=test_df.target.values,
            tokenizer=tokenizer,
            max_len=MAX_LEN
        )
        test_data_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=TEST_BATCH_SIZE,
            shuffle=True,
            drop_last=True
        )

        optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        num_training_steps = int((len(train_dataset) / TRAIN_BATCH_SIZE) * EPOCHS)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=0,
            num_training_steps=num_training_steps
        )

        for epoch in range(EPOCHS):
            train_loop_fn(train_data_loader, model, optimizer, device, scheduler)
            
        output, target = eval_loop_fn(test_data_loader, model, device)
        acc = (output.argmax(1) == target).sum() / len(target)
        print(f'Final Accuracy for Round {num}: {round(acc,4)}')
        model_stats['scores'].append(round(acc, 6))
        torch.cuda.empty_cache()
        time.sleep(3)
        print()
        print('---------------------')
    print('Logging model stats....')
    print()
    final_score = np.round(np.mean(model_stats['scores']), 4)
    model_stats['mean_score'] = final_score
    with open('logs/stats.txt', 'a') as f:
        f.write(json.dumps(model_stats))
        f.write('\n')
    print('---------------------')

/anaconda/envs/py38_pytorch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1658: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Some weights of the model checkpoint at /home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/8GB-checkpoints/run_8GB_model-trained-0-7063/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a Ber

bi=0, loss=0.715508759021759
bi=20, loss=0.3137582838535309
bi=40, loss=0.43840402364730835
bi=0, loss=0.6257165670394897
bi=20, loss=0.43836620450019836
bi=40, loss=0.625725507736206
bi=0, loss=0.5632762908935547
bi=20, loss=0.43833428621292114
bi=40, loss=0.5632702708244324
bi=0, loss=0.5008020401000977
bi=20, loss=0.5007956624031067
bi=40, loss=0.6257413625717163
bi=0, loss=0.5007944107055664
bi=20, loss=0.5007848739624023
bi=40, loss=0.5632685422897339
bi=0, loss=0.6257481575012207
bi=20, loss=0.3758203983306885
bi=40, loss=0.5632649064064026
bi=0, loss=0.5007869601249695
bi=20, loss=0.5007845163345337
bi=40, loss=0.5007820129394531
bi=0, loss=0.43829625844955444
bi=20, loss=0.43830206990242004
bi=40, loss=0.5632651448249817
Final Accuracy for Round 1: 0.6714

---------------------


Some weights of the model checkpoint at /home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/8GB-checkpoints/run_8GB_model-trained-0-7063/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint 

bi=0, loss=0.7130880355834961
bi=20, loss=0.5009768605232239
bi=40, loss=0.6256934404373169
bi=0, loss=0.4384469985961914
bi=20, loss=0.3759791851043701
bi=40, loss=0.5008022785186768
bi=0, loss=0.3759301006793976
bi=20, loss=0.4383658468723297
bi=40, loss=0.5008127689361572
bi=0, loss=0.5632714629173279
bi=20, loss=0.43834391236305237
bi=40, loss=0.5632756352424622
bi=0, loss=0.5632696747779846
bi=20, loss=0.43832123279571533
bi=40, loss=0.37585702538490295
bi=0, loss=0.3133784830570221
bi=20, loss=0.5007981061935425
bi=40, loss=0.3758460283279419
bi=0, loss=0.6882193684577942
bi=20, loss=0.43831732869148254
bi=40, loss=0.3758370578289032
bi=0, loss=0.5007929801940918
bi=20, loss=0.5007844567298889
bi=40, loss=0.4383183717727661
Final Accuracy for Round 2: 0.6714

---------------------


Some weights of the model checkpoint at /home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/8GB-checkpoints/run_8GB_model-trained-0-7063/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint 

...

bi=0, loss=0.7555574774742126
bi=20, loss=0.37812867760658264
bi=40, loss=0.37665167450904846
bi=0, loss=0.43889978528022766
bi=20, loss=0.31388285756111145
bi=40, loss=0.4385802149772644
bi=0, loss=0.31378689408302307
bi=20, loss=0.5009547472000122
bi=40, loss=0.5632438063621521
bi=0, loss=0.4382820129394531
bi=20, loss=0.5008319616317749
bi=40, loss=0.6257015466690063
bi=0, loss=0.3759329319000244
bi=20, loss=0.37595465779304504
bi=40, loss=0.5633894205093384
bi=0, loss=0.438289076089859
bi=20, loss=0.3751663863658905
bi=40, loss=0.4383544921875
bi=0, loss=0.3811506927013397
bi=20, loss=0.5319992899894714
bi=40, loss=0.41580817103385925
bi=0, loss=0.5009181499481201
bi=20, loss=0.6666535139083862
bi=40, loss=0.5196933150291443
Final Accuracy for Round 3: 0.7357

---------------------


Some weights of the model checkpoint at /home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/8GB-checkpoints/run_8GB_model-trained-22-162449/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoi

bi=0, loss=0.6158610582351685
bi=20, loss=0.43901312351226807
bi=40, loss=0.5009333491325378
bi=0, loss=0.5004693269729614
bi=20, loss=0.6257903575897217
bi=40, loss=0.5632901787757874
bi=0, loss=0.5007635354995728
bi=20, loss=0.4384327530860901
bi=40, loss=0.37599119544029236
bi=0, loss=0.5008662939071655
bi=20, loss=0.438281387090683
bi=40, loss=0.5006917119026184
bi=0, loss=0.43825563788414
bi=20, loss=0.5630446672439575
bi=40, loss=0.4383472800254822
bi=0, loss=0.5632711052894592
bi=20, loss=0.3758915662765503
bi=40, loss=0.5008163452148438
bi=0, loss=0.6257544159889221
bi=20, loss=0.5008159875869751
bi=40, loss=0.37587666511535645
bi=0, loss=0.500870943069458
bi=20, loss=0.375882089138031
bi=40, loss=0.5008062124252319
Final Accuracy for Round 4: 0.6714

---------------------


Some weights of the model checkpoint at /home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/8GB-checkpoints/run_8GB_model-trained-22-162449/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoi

bi=0, loss=0.6855602264404297
bi=20, loss=0.4389772117137909
bi=40, loss=0.6880782842636108
bi=0, loss=0.43871018290519714
bi=20, loss=0.43851858377456665
bi=40, loss=0.3136983811855316
bi=0, loss=0.43845421075820923
bi=20, loss=0.37621986865997314
bi=40, loss=0.5008324384689331
bi=0, loss=0.37800633907318115
bi=20, loss=0.35351115465164185
bi=40, loss=0.5044620037078857
bi=0, loss=0.43866071105003357
bi=20, loss=0.5009447932243347
bi=40, loss=0.6228271722793579
bi=0, loss=0.43299248814582825
bi=20, loss=0.563065230846405
bi=40, loss=0.4390726089477539
bi=0, loss=0.563274621963501
bi=20, loss=0.43844789266586304
bi=40, loss=0.5008668303489685
bi=0, loss=0.4383482038974762
bi=20, loss=0.5011698603630066
bi=40, loss=0.4106025695800781
Final Accuracy for Round 5: 0.6714

---------------------
Logging model stats....

---------------------


In [13]:
torch.cuda.empty_cache()